In [1]:
import json
import pickle
import operator
import json
from collections import Counter, defaultdict
import pandas as pd
import numpy as np

## Load files

In [3]:
aussie_items = json.load(open('./data/fixed/australian_users_items_fixed.json','r'))
steam_games = json.load(open('./data/fixed/steam_games_fixed.json','r'))

## Create User-Item Matrix

In [4]:
def build_list(user_item_data):
    user_item = []
    for user in user_item_data:
        for item in user['items']:
            user_item.append((user['user_id'],item['item_name']))
    return user_item

In [5]:
def build_df(user_item):
    df = pd.DataFrame(user_item,columns=['user','item'])
    df = df.drop_duplicates(['user','item'])
    df['own'] = 1
    df = df.pivot(index='user',columns='item',values='own')
    df = df.fillna(0)
    return df

In [6]:
user_item = build_list(aussie_items)
user_item_df = build_df(user_item)

## Filter game functions

In [8]:
## Filter by owned by percentage of all users

def filter_percentage(user_item, percentage=0.02):
    item_count = [i[1] for i in user_item]
    user_count = len({i[0] for i in user_item})
    item_percentages = {k:v/user_count for k,v in Counter(item_count).items()}
    filtered_games = [k for k,v in item_percentages.items() if v > percentage]
    filtered_user_item = [i for i in user_item if i[1] in filtered_games]
    return filtered_user_item

In [9]:
## Filter by most owned games

def filter_top_n(user_item, top_n=500):
    item_count = [i[1] for i in user_item]
    item_sum = sorted({k:v for k,v in Counter(item_count).items()}.items(), key=operator.itemgetter(1), reverse=True)
    filtered_games = [i[0] for i in item_sum[0:top_n]]
    filtered_user_item = [i for i in user_item if i[1] in filtered_games]
    return filtered_user_item

## Save User_Item list

In [14]:
user_item_filter_t1000 = filter_top_n(user_item,1000)
with open('./data/saved/user_item_filter_t1000.json','w') as file:
    json.dump(user_item_filter_t1000, file)

In [13]:
user_item_filter_t500 = filter_top_n(user_item,500)
with open('./data/saved/user_item_filter_t500.json','w') as file:
    json.dump(user_item_filter_t500, file)